In [2]:
import json
import os
import pandas as pd
import numpy as np
import random
from itertools import product

In [ ]:


def generate_triggers(condition_dict):
    """
    Generate a dictionary of trigger definitions based on combinations of condition levels.

    Parameters:
        condition_dict (dict): A dictionary where keys are factor names (e.g., 'v_stimulus')
                               and values are lists of condition levels for that factor.

    Returns:
        triggers (dict): A dictionary with trial_type strings as keys, and sub-dictionaries
                         containing trigger_id and event codes.
        num_triggers (int): Total number of unique trial combinations.
    """
    # Extract all factor names and corresponding level lists
    factor_names = list(condition_dict.keys())
    level_lists = list(condition_dict.values())

    # Compute all combinations (Cartesian product)
    combinations = list(product(*level_lists))
    num_triggers = len(combinations)
    
    triggers = {}
    for i, levels in enumerate(combinations, start=1):
        # Create trial_type string
        trial_type = "_".join(str(level) for level in levels)
        
        # Define sub-dictionary with trigger structure
        triggers[trial_type] = {
            "trial_start": i,
            "cue_onset": i + num_triggers,
            "isi": i + 2 * num_triggers,
            "target_onset": i + 3 * num_triggers,
            "response_onset": i + 4 * num_triggers,
        }

    return triggers


In [36]:
# Define conditions
conditions = {
    "v_stimulus": [45, 135],
    "v_pred_cond": ["EXP", "UEX", "neutral"],
    "a_stimulus": [100, 160],
    "a_pred_cond": ["EXP", "UEX"],
}
# Generate triggers
triggers = generate_triggers(conditions)
# Convert nested dictionary to DataFrame with trial_type as the index
triggers_df = pd.DataFrame.from_dict(triggers, orient='index')
triggers_df.to_csv("triggers.csv", index=True, header=True)

In [1]:
def generate_triggers(condition_dict):
    """
    Generate a dictionary of trigger definitions based on combinations of condition levels.

    Parameters:
        condition_dict (dict): A dictionary where keys are factor names (e.g., 'v_stimulus')
                               and values are lists of condition levels for that factor.

    Returns:
        triggers (dict): A dictionary with trial_type + event as strings as keys and trigger numbers as values.
    """
    # ===== TRIGGERS FOR LERNING AND TEST PHASES =====
    # Extract all factor names and corresponding level lists
    level_lists = list(condition_dict.values())

    # Compute all combinations (Cartesian product)
    combinations = list(product(*level_lists))
    num_triggers = len(combinations)
    
    triggers = {}
    for i, levels in enumerate(combinations, start=1):
        trial_type = "_".join(str(level) for level in levels)
    
        # Define sub-dictionary with trigger structure
        triggers[f"{trial_type}_trial_start"] = i 
        triggers[f"{trial_type}_cue_onset"] = i + num_triggers
        triggers[f"{trial_type}_isi"] = i + 2 * num_triggers
        triggers[f"{trial_type}_target_onset"] = i + 3 * num_triggers
        triggers[f"{trial_type}_response"] = i + 4 * num_triggers

    # ===== TRIGGERS FOR EXPLICIT PHASE =====
    # visual block conditions
    visual_explicit_conditions = {
        "v_stimulus": ["45", "135"],
        "v_pred_cond": ["EXP", "UEX"],
        }
    # auditory block conditions
    auditory_explicit_conditions = {
        "a_stimulus": ["100", "160"],
        "a_pred_cond": ["EXP", "UEX"],
        }
    
    # Adding visual explicit triggers
    # Extract all factor names and corresponding level lists
    visual_level_lists = list(visual_explicit_conditions.values())
    # Compute all combinations (Cartesian product)  
    visual_combinations = list(product(*visual_level_lists))
    visual_num_triggers = len(visual_combinations)

    for i, levels in enumerate(visual_combinations, start= max(triggers.values()) + 1): 
        trial_type = "_".join(str(level) for level in levels)
    
        # Define sub-dictionary with trigger structure
        triggers[f"explicit_{trial_type}_trial_start"] = i 
        triggers[f"explicit_{trial_type}_cue_onset"] = i + visual_num_triggers
        triggers[f"explicit_{trial_type}_isi"] = i + 2 * visual_num_triggers
        triggers[f"explicit_{trial_type}_target_onset"] = i + 3 * visual_num_triggers
        triggers[f"explicit_{trial_type}_response"] = i + 4 * visual_num_triggers
        triggers[f"explicit_{trial_type}_confidence"] = i + 5 * visual_num_triggers


    # Adding auditory explicit triggers
    # Extract all factor names and corresponding level lists
    auditory_level_lists = list(auditory_explicit_conditions.values())
    # Compute all combinations (Cartesian product)  
    auditory_combinations = list(product(*auditory_level_lists))
    auditory_num_triggers = len(auditory_combinations)

    for i, levels in enumerate(auditory_combinations, start= max(triggers.values()) + 1):
        trial_type = "_".join(str(level) for level in levels)

        # Define sub-dictionary with trigger structure
        triggers[f"explicit_{trial_type}_trial_start"] = i
        triggers[f"explicit_{trial_type}_cue_onset"] = i + auditory_num_triggers
        triggers[f"explicit_{trial_type}_isi"] = i + 2 * auditory_num_triggers
        triggers[f"explicit_{trial_type}_target_onset"] = i + 3 * auditory_num_triggers
        triggers[f"explicit_{trial_type}_response"] = i + 4 * auditory_num_triggers
        triggers[f"explicit_{trial_type}_confidence"] = i + 5 * auditory_num_triggers
    

    # ===== TRIGGERS FOR LOCALIZER PHASE =====
    highest_trigger = max(triggers.values()) # Get the highest trigger number
    localizer_triggers = {
    "loc_start": highest_trigger + 1, 
    "loc_isi": highest_trigger + 2,
    "loc_45_100": highest_trigger + 3,
    "loc_45_160": highest_trigger + 4,
    "loc_135_100": highest_trigger + 5,
    "loc_135_160": highest_trigger + 6,
    "loc_response": highest_trigger + 7,
    }
    
    return triggers | localizer_triggers # Merge dictionaries 

In [3]:
# Define conditions
conditions = {
    "v_stimulus": [45, 135],
    "v_pred_cond": ["EXP", "UEX", "neutral"],
    "a_stimulus": [100, 160],
    "a_pred_cond": ["EXP", "UEX"],
}
# Generate triggers
triggers = generate_triggers(conditions)

In [4]:
triggers

{'45_EXP_100_EXP_trial_start': 1,
 '45_EXP_100_EXP_cue_onset': 25,
 '45_EXP_100_EXP_isi': 49,
 '45_EXP_100_EXP_target_onset': 73,
 '45_EXP_100_EXP_response': 97,
 '45_EXP_100_UEX_trial_start': 2,
 '45_EXP_100_UEX_cue_onset': 26,
 '45_EXP_100_UEX_isi': 50,
 '45_EXP_100_UEX_target_onset': 74,
 '45_EXP_100_UEX_response': 98,
 '45_EXP_160_EXP_trial_start': 3,
 '45_EXP_160_EXP_cue_onset': 27,
 '45_EXP_160_EXP_isi': 51,
 '45_EXP_160_EXP_target_onset': 75,
 '45_EXP_160_EXP_response': 99,
 '45_EXP_160_UEX_trial_start': 4,
 '45_EXP_160_UEX_cue_onset': 28,
 '45_EXP_160_UEX_isi': 52,
 '45_EXP_160_UEX_target_onset': 76,
 '45_EXP_160_UEX_response': 100,
 '45_UEX_100_EXP_trial_start': 5,
 '45_UEX_100_EXP_cue_onset': 29,
 '45_UEX_100_EXP_isi': 53,
 '45_UEX_100_EXP_target_onset': 77,
 '45_UEX_100_EXP_response': 101,
 '45_UEX_100_UEX_trial_start': 6,
 '45_UEX_100_UEX_cue_onset': 30,
 '45_UEX_100_UEX_isi': 54,
 '45_UEX_100_UEX_target_onset': 78,
 '45_UEX_100_UEX_response': 102,
 '45_UEX_160_EXP_trial_st

In [24]:
highest_trigger = max(triggers.values()) # Get the highest trigger number 

TRIGGERS_LOCALIZER = {
    "loc_start": highest_trigger + 1, 
    "45_100": highest_trigger + 2,
    "45_160": highest_trigger + 3,
    "135_100": highest_trigger + 4,
    "135_160": highest_trigger + 5,
}

In [26]:
triggers | TRIGGERS_LOCALIZER

{'45_EXP_100_EXP_trial_start': 1,
 '45_EXP_100_EXP_cue_onset': 25,
 '45_EXP_100_EXP_isi': 49,
 '45_EXP_100_EXP_target_onset': 73,
 '45_EXP_100_EXP_response': 97,
 '45_EXP_100_UEX_trial_start': 2,
 '45_EXP_100_UEX_cue_onset': 26,
 '45_EXP_100_UEX_isi': 50,
 '45_EXP_100_UEX_target_onset': 74,
 '45_EXP_100_UEX_response': 98,
 '45_EXP_160_EXP_trial_start': 3,
 '45_EXP_160_EXP_cue_onset': 27,
 '45_EXP_160_EXP_isi': 51,
 '45_EXP_160_EXP_target_onset': 75,
 '45_EXP_160_EXP_response': 99,
 '45_EXP_160_UEX_trial_start': 4,
 '45_EXP_160_UEX_cue_onset': 28,
 '45_EXP_160_UEX_isi': 52,
 '45_EXP_160_UEX_target_onset': 76,
 '45_EXP_160_UEX_response': 100,
 '45_UEX_100_EXP_trial_start': 5,
 '45_UEX_100_EXP_cue_onset': 29,
 '45_UEX_100_EXP_isi': 53,
 '45_UEX_100_EXP_target_onset': 77,
 '45_UEX_100_EXP_response': 101,
 '45_UEX_100_UEX_trial_start': 6,
 '45_UEX_100_UEX_cue_onset': 30,
 '45_UEX_100_UEX_isi': 54,
 '45_UEX_100_UEX_target_onset': 78,
 '45_UEX_100_UEX_response': 102,
 '45_UEX_160_EXP_trial_st

In [5]:
stimuli = [(45, 100), (45, 160), (135, 100), (135, 160)] * 2
random.shuffle(stimuli)
stimuli

[(45, 100),
 (135, 160),
 (135, 100),
 (45, 100),
 (135, 160),
 (135, 100),
 (45, 160),
 (45, 160)]

In [6]:
print([pair[0] for pair in stimuli])
print([pair[1] for pair in stimuli])

[45, 135, 135, 45, 135, 135, 45, 45]
[100, 160, 100, 100, 160, 100, 160, 160]


In [7]:
[0] * 8

[0, 0, 0, 0, 0, 0, 0, 0]